# Introduction
# cancelation cancelation cancelation
this is one of the biggest problems that faces the tourism industry which make this industry more risky than other investments , predicting the real demand is a real challenge for the managers in this field and helps them improve their profits, decrease the risk, and be always ready with enough facilities

This dataset contains two datasets with hotel demand data. One of the hotels (H1) is a resort hotel and the other is a city hotel (H2). Both datasets share the same structure, with 31 variables describing the 40,060 observations of H1 and 79,330 observations of H2. Each observation represents a hotel booking. Both datasets comprehend bookings due to arrive between the 1st of July of 2015 and the 31st of August 2017, including bookings that effectively arrived and bookings that were canceled. Both hotels are located in Portugal: H1 at the resort region of Algarve and H2 at the city of Lisbon
our main aimfrom this project is to discover the data ,knowing the distributions of our features and try to discover the realationships between them focusing the problem of high_cancelation_rate in order to find advices to decrease this rate or even predict it accordings the different features

In [ ]:
#importing necessary libraries ans loading the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#loading the data
hotels=pd.read_csv(r'../input/hotel-booking-demand/hotel_bookings.csv')

In [ ]:
#taking a look
hotels.head()

In [ ]:
#a vertical view for the data
hotels.sample(3).T

In [ ]:
#checking the nulls
hotels.isnull().sum()

In [ ]:
#checking the unique values of our data
hotels.nunique()

In [ ]:
#checking the duplicates
hotels.duplicated().sum()

In [ ]:
#checking some descriptive statistics of the numeric variables
hotels.describe()

## it seems that we have sme outliers observations in the data specially in adr so we have to check

In [ ]:
#checking for outliers
hotels.adr.nlargest()

In [ ]:
hotels.babies.nlargest()

In [ ]:
hotels.stays_in_week_nights.nlargest()

In [ ]:
hotels.adults.nlargest(20)

In [ ]:
hotels.children.nlargest()

# we can note that:   
> - we have two features that miss alot of data 'company' & 'agent'  
> - some other features that have few unique values would be better to convert their type to   __category__  
> -  we have a big number of __duplicates__ in our data __but__  whereas the variables don't include any unique identification column or names and this data is pr# we can note that:   
> - we have two features that miss alot of data 'company' & 'agent'  
> - some other features that have few unique values would be better to convert their type to   __category__  
> -  we have a big number of __duplicates__ in our data __but__  whereas the variables don't include any unique identification column or names and this data is professionally gathered which dosen't allow this huge number of duplicates so we can freely assume that this observations are just repeated observations for different guests or groups     
   


 # Data wrangling

In [ ]:
#dropping the columns with missing values
hotels.drop(['company','agent'],axis=1,inplace=True)

In [ ]:
#converting the type of some columns to categorical
hotels[['meal','arrival_date_month','deposit_type','reservation_status','market_segment','distribution_channel',"is_canceled"]]=hotels[['meal','arrival_date_month','deposit_type','reservation_status','market_segment','distribution_channel',"is_canceled"]].astype('category')

In [ ]:
#renaming and reordering the cancelation values
hotels.is_canceled.cat.rename_categories(['confirmed','canceled'],inplace=True)
hotels.is_canceled.cat.reorder_categories(['confirmed','canceled'],inplace=True)

In [ ]:
#creating the total _nights feature
hotels['total_nights']=hotels.stays_in_weekend_nights+hotels.stays_in_week_nights

In [ ]:
#filling the few missing values of country coloumn
hotels.country.fillna(method='ffill',inplace=True)

In [ ]:
#dropping the outlier point from the data
outlier=hotels[(hotels.adr==5400)|(hotels.children==10)|(hotels.babies>5)|(hotels.adults>4)|(hotels.total_nights>30)].index
hotels.drop(outlier,axis=0,inplace=True)

In [ ]:
# dropping 4 rows with nan values in choldren coloumn
hotels.dropna(axis='rows',inplace=True)

# rechicking the data det after wrangling

In [ ]:
#recheckingthe data
hotels.info()

In [ ]:
#some descriptive statistics about the numeric variables in dataset after dropping outliers
hotels.describe()

# data is clean and ready to get discovered

In [ ]:
#preparing for plotting
sns.set()
color=sns.color_palette()[0]
sns.set_style('whitegrid')
sns.set_context('talk')
palette='PuBuGn_r'

In [ ]:
#the propotion of each hotel kind that has been reserved
hotels.hotel.value_counts(normalize=True)

> __two third of the reservation was for the city hote and one third fot the resort hotel__

> __let's check the cancelation ratio of each hotel__ 

In [ ]:
hotels.is_canceled.value_counts(normalize=True)

In [ ]:
#cancelation average for city hotel 
hotels[hotels.hotel=='City Hotel'].is_canceled.value_counts(normalize=True)

In [ ]:
#cancelation average of resort hotel
hotels[hotels.hotel=='Resort Hotel'].is_canceled.value_counts(normalize=True)

__the overall cancelation rate is 37% and this percentage is realy big and need to deep understand to get good predicted  
this percentage is differ between the two hotels , the city hotel has around 1.5 times worser rate__

# some univariate visualizations to recognize the distribution of features 

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
hotels.hotel.value_counts().plot.pie(explode=(0,.05),startangle=90,autopct='%1.1f%%')
plt.title('booking propotion')
plt.subplot(1,2,2)
hotels[hotels.is_canceled=='confirmed'].hotel.value_counts().plot.pie(explode=(0,.05),startangle=90,autopct='%1.1f%%')
plt.title('distribution of the total cancelation propotion')

__city hotel is most booked with 66.5% from all bookings against 35.5% for resort hotel but also it has the biggest percentage of the total cancelation (74.5%)__ 

# let's check the distribution of some numeric features

In [ ]:
plt.figure(figsize=(8,8))
sns.histplot(hotels.adr,kde=True,discrete=True)
plt.xlim(0,250)
plt.title('distribution of the average dialy rate')

In [ ]:
plt.figure(figsize=(8,8))
sns.histplot(hotels.lead_time,kde=True)
plt.title('the distribution of the lead_time')

In [ ]:
plt.figure(figsize=(8,8))
sns.histplot(hotels.total_nights,color=color)
plt.xlim(0,17)
plt.title('distribution of total_nights booked')

- __the three plots above say that the distribution of the ADR is multimodal with multi peaks   can also consider as right skewed distribution which is the favorite for the investorsin the long_term investment__

- __while the distributions of the lead_time and the total_nights are straight and clearly positive skewed (the mean is right of the peak)__****

# some conditional plots makes the image more obvious

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=hotels,x="is_canceled", hue='hotel',bins=5,stat='frequency',multiple='dodge',element='bars',palette='Blues',shrink=.8)
plt.title('the bookings be canceled or confirmed')

__from the graph above we can recognize that the cancelation is real problem specially for city hotel__

# what about the the monthes , do we have to predict more cancelations in specific monthes or seasons .. let's discover the distribution of the bookings throw the different monthes

In [ ]:
#rordering the monthes category for better plotting
cat=['January','February','March','April','May','June','July','August','September','October','November','December']
hotels['arrival_date_month']=hotels.arrival_date_month.cat.reorder_categories(cat)
sns.catplot(data=hotels,x='arrival_date_month',hue='is_canceled',palette=palette,kind='count',height=6)
plt.xticks(rotation=45)
plt.title('cancelation in different monthes of the year')

__its clear that august and julie__ are our top_season when we expect the double number of guests  than other monthes like january and december but they are also the **top_season of the cancelation**

# what about the ADR the (the daily average rate) what or how much can we expect in the different times of the year

In [ ]:
plt.figure(figsize=(8,6))
plt.title('the estimated daily average rate per occupied room with standars_deviation per month')
plt.xticks(rotation=45)
sns.barplot(data=hotels,x='arrival_date_month',y='adr',hue='hotel',palette=palette,ci='sd')

> - __as expected the ADR of the top_season_monthes is the greatest__

In [ ]:
#plotting boxplots for ADR in the different years
plt.figure(figsize=(8,6))
sns.violinplot(data=hotels,y='adr',x='arrival_date_year',hue='hotel',palette= 'BuPu' ,split=True,inner='quartile')
plt.ylabel('AVERAGE DAILY RATE')
plt.xlabel('YEAR OF ARRIVAL')
plt.title('the daily average of the different years')

> __even with the high rate of cancelation__ in augest an july but both hotels can expect to get their best rate in these monthes   

# let's go on discovering the relationdhip between cancelations and other features

In [ ]:
sns.catplot(data=hotels,x='distribution_channel',hue='is_canceled',palette=palette,kind='count',height=6)
plt.title('distribution of distribution channel according cancelation')

> __travel agents & tour operators are have the biggest reservation ratio but also the biggest cancelation ratio while the direct bookings has
the best cancelation ratio__ 

In [ ]:
sns.catplot(data=hotels,x='market_segment',hue='is_canceled',palette=palette,kind='count',height=6)
plt.xticks(rotation=45)
plt.title('the market segments and cancelations')

> - __online travel_agents are th bigest source for the reservations of our destination, while the direct_reservation has the least cancelation ratio__

In [ ]:
sns.catplot(data=hotels,x='deposit_type',hue='is_canceled',palette=palette,kind='count',height=6)
plt.title('distribution of deposit type according cancelation')

> - __booking with no deposit is the most common type__
> - __almost all of nun_refund deposit type reservation were canceled that was totaly unexpected__ 

# what is the most common type of the customers ?

In [ ]:
g=sns.catplot(data=hotels,x='customer_type',hue='is_canceled',palette=palette,kind='count',height=6)
plt.xticks(rotation=45)

> - __the most common type of customers is the transient travelers that's may be why the most of bookings are from one to three nights__

# which hotel has the higher ADR ?

In [ ]:
#the distribution of the average daily rate of both hotels
sns.displot(x=hotels.adr,height=6,hue=hotels.hotel,kind='kde')
plt.title('the distribution of the average daily rate of both hotels')
plt.xlim(0,400)

> __it's clear that the distribution of ADR is right skewed and the resort hotel has better rate__

In [ ]:
sns.displot(x=hotels.lead_time,kind='kde', hue=hotels.is_canceled,fill=True,height=5.5)
plt.title('density distribution of the lead_time')
plt.xlim(0,500)

> __it's a positive skewed distribution the mean lead_time is greater than the most common lead_time__

# which country book the destination the most ?

In [ ]:
countries=hotels.country.value_counts().nlargest(10)
plt.figure(figsize=(8,6))
sns.barplot(y=countries.index, x=countries.values, alpha=0.8)
plt.title('the destination most booking countries')

> __the most pepole reserving the destination are native citizens__

# is there a relationship between the previous cancelation of the customer and the probability that he cancel again ?

In [ ]:
sns.jointplot(data=hotels,x='is_canceled',y='previous_cancellations',height=6)

- __we see from this graph that the pepole how had more than 6 previous cancelations will often cancel their future reseravtion
thoese pepole may be not serious with their reservations__

# what about the total nights booked and the ADR and the probability of the reservation to be canceled ?

In [ ]:
sns.jointplot(data=hotels,x='adr',y='total_nights',hue='is_canceled',height=7,marker='.',s=100)
plt.title('total_nights , ADR and cancelation probability')

- __from the graph above we see that reservations with middle daily average rate have more chance to be canceled specially these reservations of more than 15 nights__

# let's make a simple prediction model  

In [ ]:
#preparing our features for the model 
hotels.reset_index()
hotels_cat=pd.get_dummies(hotels[['hotel','meal','is_repeated_guest','market_segment','customer_type','arrival_date_week_number','country','reserved_room_type']])
hotels_num= hotels[['total_nights','adults','children','previous_cancellations','lead_time']]
X=hotels_cat.join(hotels_num,how='inner')
y=hotels.is_canceled

In [ ]:
#fitting the model
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=.1)
clf=LogisticRegression(C=1000,solver='sag',max_iter=10000)
clf.fit(x_train,y_train)

In [ ]:
#model accurancy score
print('the accuracy score of the prediction model is:  ',clf.score(x_test,y_test))

# let's plot the the relationship between some important features

In [ ]:
g = sns.PairGrid(hotels[['is_canceled','lead_time','distribution_channel','deposit_type','previous_cancellations','adr','arrival_date_day_of_month','total_nights']])
g.map(sns.scatterplot,alpha=.5)

# we can see the correlations  between the numeric features

In [ ]:
corr=hotels.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# conclusions

> - __the cancelation and its prediction is a real problem for the tourism industry and good understandig for this problem and the features that related with will be very useful to decrease the investments's risk of this important industry__
> - __deep understanding of these relationships between cancelations and any features that related with will help managers to improve the confirmation rate of their reservations__
> __from the dataset we could find some possible relationships between the cancelations and some features even they are not strong or certain__ 
- there is a possible relationship between the customers with previous cancelation and the future cancelations 
- the cancelations increses when the adr is in its common mode specially when the total nights booked are more than 15 nights 
- we couldn't find any clear and strong realationship between any of the features and the cancelation may be we have to include more factors in the dataset we can set a question in cancelation form asking customers to set a reason , that will be more helpful 
- by deeper research we can build a stronger model in future to predict the actual confirmed reservations which will help the managers be ready for the real demand 
- by deeper research we can advice the managers to improve choosing their agents and working together with them to improve the occupation of their hotels
- most customers are from Portugal , nice but the managers must give extra attention for the international advertising and international distribution channels

Thanks for sharing dataset